In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
#import uproot
import uproot3
from scipy import optimize as op
import scipy.integrate as integrate

In [2]:
#Gain access to functions used to change some of the aspects regarding arrays and the things in them
%run /Users/cmc/analysis_phd/analysis_twopi_clas6/cut_determinations/e16/Bin\ Conversions.ipynb

#Gain Access to functions used to work with momentum binning
%run /Users/cmc/analysis_phd/analysis_twopi_clas6/cut_determinations/e16/Momentum\ Bins.ipynb

#Gain Access to functions used to work with Fitting Functions
%run /Users/cmc/analysis_phd/analysis_twopi_clas6/cut_determinations/e16/Fitting\ Functions.ipynb

#Gain Access to functions used for TH2s 
#uproot or uproo4
%run /Users/cmc/analysis_phd/analysis_twopi_clas6/cut_determinations/e16/TH2\ Functions-uproot3.ipynb
#uproot3
#%run /Users/cmc/analysis_phd/analysis_twopi_clas6/cut_determinations/e16/TH1\ Functions-uproot3.ipynb

#Gain Access to functions for taking discrete derivatives of lists of data
%run /Users/cmc/analysis_phd/analysis_twopi_clas6/cut_determinations/e16/Derivatives.ipynb

*Bin Conversions*
Available Functions:
Note: All functions return new arrays rather than altering the previous array
	New_Merged_Size(array_,merge_size_) returns what the size of the new merged array will be
	Merge_Bin_Edges(edges_,merge_size) merges the bin edges 
	Merge_Bins(array_,merge_size_) adds together every <merge_size_> values in an array
	Merge_Vals(yarray_,merge_size_) adds together every <merge_size_> ybin together in an array
	Merge_Bins(xarray_,merge_size_) averages every <merge_size_> xbins together in an array
	Count_Non_Zero(array_) counts the number of nonzero values in an array
	Count_Zero(array_) counts the number of zeros in an array
	Remove_Zeros(array_) remove zeros from an array
	Remove_Zeros_Y(yarray_) removes zero values from yarray
	Remove_Zeros_X(xarray_,yarray_) removes xbins where the corresponding y value was zero
	Count_InBounds(yarray_,low_,high_) counts the number of values that fall between bounds (or equal to)
	Count_OutBounds(yarray_,low_,high_) co

In [3]:
#date="05-07-2024"
#dir="/Users/cmc/analysis_phd/root_files/cut_determination/cut_det_e16/"+date
#dir="/Users/cmc/analysis_phd/root_files/cut_determination/cut_det_e16/"+date+"/cc"
dir="/Users/cmc/analysis_phd/root_files/local_runs/e16/target"
root_file=dir+"/analysis_full_e16_geo_fid.root"
#root_file=dir+"/exp_e16_clas6_fid_geo_"+date+".root"
file=uproot3.open(root_file)

species="pip"
detector="sc"
cut="pid"

sector_names = ["sec1","sec2","sec3","sec4","sec5","sec6","all"]
side = ["left","coinc","right","All"]
num_cc_seg = 18
num_cc_side = 3
num_sc_pad = 48
#0-> pid | 1->event selection all 

In [4]:
layer1="Geometric Distributions"
layer2="Geo Fid "+species
layer3=layer2+" "+detector
layer4=[layer3+" "+sector_names[sec] for sec in range(7)]
layer5=[layer4[sec]+" "+cut for sec in range(7)]

    

In [5]:
hist_idx_len = len(file[layer1+"/"+layer2+"/"+layer3+"/"+layer4[0]+"/"+layer5[0]].keys())

hist = [[file[layer1+"/"+layer2+"/"+layer3+"/"+layer4[sec]+"/"+layer5[sec]][file[layer1+"/"+layer2+"/"+layer3+"/"+layer4[sec]+"/"+layer5[sec]].keys()[2*hist_idx]] for hist_idx in range(round(hist_idx_len/2))] for sec in range(7)]


cut_hist_len = round(hist_idx_len/4)

In [6]:
def x_idx(xarray_, xval_):
    for i in range(len(xarray_)):
        if xarray_[i] <= xval_:
            if xarray_[i+1] >= xval_:
                return i
    return -1

def y_idx(yarray_, yval_):
    for i in range(len(yarray_)):
        if yarray_[i] <= yval_:
            if yarray_[i+1] >= yval_:
                return i
    return -1

In [7]:
#For cut only
def side_idx(hist_idx_):
    if hist_idx_ < num_cc_seg*3:
        if hist_idx_ < num_cc_seg:
            return 0
        elif hist_idx < num_cc_seg*2:
            return 1
        else:
            return 2
    elif hist_idx_ <= num_cc_seg*3+3:
        return hist_idx_ - (num_cc_seg*3)

def seg_idx(hist_idx_):
    if hist_idx_ < num_cc_seg*3:
        if hist_idx_ < num_cc_seg:
            return hist_idx_
        elif hist_idx < num_cc_seg*2:
            return hist_idx_ - num_cc_seg
        else:
            return hist_idx_ - num_cc_seg*2
    elif hist_idx_ <= num_cc_seg*3+3:
        return _num_cc_seg
    
def pad_idx(hist_idx_):
    if hist_idx_ <= _num_sc_pad:
        return hist_idx_ 
    else:
        return -1
    
def cc_hist_idx(side_, seg_):
    if side_ == 3:
        if seg_ == num_cc_seg:
            return cut_hist_len-1
        else:
            return -1
    else:
        if seg_ == num_cc_seg:
            return num_cc_seg*3 + side_
        else:
            return num_cc_seg*side_ + seg_


In [8]:
#Look at Overall Distributions
xdim_len = len(Center_Xbins(hist[0][0]))
ydim_len = len(Center_Ybins(hist[0][0]))

x_merge = 1
y_merge = 1


xvals = Merge_Bins(Center_Xbins(hist[0][0]),x_merge)
yvals = Merge_Bins(Center_Ybins(hist[0][0]),y_merge)
zvals_xy = [[Merge_Z(XY_Values(hist[sec][hist_idx]),x_merge,y_merge) for hist_idx in range(cut_hist_len)] for sec in range(7)]
zvals_yx = [[Merge_Z(YX_Values(hist[sec][hist_idx]),y_merge,x_merge) for hist_idx in range(cut_hist_len)] for sec in range(7)]

xdim = len(xvals)
ydim = len(yvals)


In [ ]:
dumb1_x= np.zeros((7,cut_hist_len,xdim*ydim))
dumb1_y= np.zeros((7,cut_hist_len,xdim*ydim))
dense1 = np.zeros((7,cut_hist_len,xdim*ydim))
dumb2_x= np.zeros((7,cut_hist_len,xdim*ydim))
dumb2_y= np.zeros((7,cut_hist_len,xdim*ydim))
dense2 = np.zeros((7,cut_hist_len,xdim*ydim))
for sec in range(7):
    for i in range(xdim):
        for j in range(ydim):
            for hist_idx in range(cut_hist_len):
                dumb1_x[sec][hist_idx][i+j*xdim]+=xvals[i]
                dumb1_y[sec][hist_idx][i+j*xdim]+=yvals[j]
                dense1[sec][hist_idx][i+j*xdim]+=zvals_xy[sec][hist_idx][i][j]
                dumb2_x[sec][hist_idx][j+i*ydim]+=yvals[j]
                dumb2_y[sec][hist_idx][j+i*ydim]+=xvals[i]
                dense2[sec][hist_idx][j+i*ydim]+=zvals_yx[sec][hist_idx][j][i]
for sec in range(6):
    plt.hist2d(dumb1_x[sec][cut_hist_len-1], dumb1_y[sec][cut_hist_len-1], bins=(xdim, ydim), weights=dense1[sec][cut_hist_len-1])
    plt.title("Pi+ SC Sector Centered Distribution Sec:{} All Segments and Sides".format(sec+1))
    plt.xlabel("Distance from Center of Detector (cm)")
    plt.ylabel("Distance from the Beamline (cm)")
    plt.show()

In [ ]:
proj_y = np.zeros((7,cut_hist_len,ydim))
for sec in range(7):
    for i in range(ydim):
        for j in range(xdim):
            for hist_idx in range(cut_hist_len):
                proj_y[sec][hist_idx][i] += zvals_xy[sec][hist_idx][j][i]

In [ ]:
sec = 0
hist_idx = 3
print(np.argmax(proj_y[sec][hist_idx]))
print(proj_y[sec][hist_idx][np.argmax(proj_y[sec][hist_idx])])

In [ ]:
def super_gaus_cut_bound(frac_,side_,a_,m_,s_,p_):
    if side_ == 0:
        return 0.5*(2*m_ - 2 * (math.log(1./frac_)**(1./p_))*(2**(p_/2.)*(1./s_)**(-p_))**(1./p_))
    elif side_ ==1:
        return 0.5*(2*m_ + 2 * (math.log(1./frac_)**(1./p_))*(2**(p_/2.)*(1./s_)**(-p_))**(1./p_))
    else:
        return -99.9
    

def Sup_Gaus_Cutoff(x_,frac_,a_,m_,s_,p_):
    if x < super_gaus_cut_bound(frac_,0,a_,m_,s_,p_):
        return 0.
    elif x > super_gaus_cut_bound(frac_,1,a_,m_,s_,p_):
        return 0.
    else:
        return Super_Gaus(x_,a_,m_,s_,p_)
    
def Sup_Gaus_Cutoff_Array(x_,frac_,a_,m_,s_,p_):
    output = np.zeros(len(x_))
    for i in range(len(x_)):
        output[i] = Sup_Gaus_Cutoff(x_[i],frac_,a_,m_,s_,p_)
    return output

In [ ]:
cut_lines = np.zeros((7,cut_hist_len,2))

low = 0.0
high = 0.0
space = 15

#Super_Gaus(x,a_,m_,s_,p_)


fit_pars = np.zeros((7,cut_hist_len,4))

ini_par = [250.0,75.0,5.4,4.0]

test_pars = np.zeros(4)

frac_cut = 0.1

paddle_expand = 19


for sec in range(7):
    for i in range(4):
        test_pars[i] = ini_par[i]
    for hist_idx in range(cut_hist_len):
        if(Count_Non_Zero(proj_y[sec][hist_idx]) >= 6):
            if hist_idx == num_sc_pad:
                low = 60.
                high = 400.
            else:
                low = yvals[np.argmax(proj_y[sec][hist_idx])] - space
                high = yvals[np.argmax(proj_y[sec][hist_idx])] + space
            new_yvals = Bounds_on_Y(yvals,low,high)
            new_zvals = Bounds_on_X_From_Y(proj_y[sec][hist_idx],yvals,low,high)
            plt.scatter(new_yvals,new_zvals,label="Data")
            if hist_idx < num_sc_pad:
                try: 
                    pars, pars_cov = op.curve_fit(Super_Gaus,new_yvals,new_zvals,p0=test_pars,maxfev=5000)
                    print("Parameters for Sec:{} Paddle:{}".format(sec+1,hist_idx+1))
                    print("test pars",test_pars)
                    print("fit pars",pars)
                    cont_x = np.linspace(new_yvals[0],new_yvals[len(new_yvals)-1],200)
                    plt.plot(cont_x,Super_Gaus(cont_x,*test_pars),label="ini_pars")
                    plt.plot(cont_x,Super_Gaus(cont_x,*pars),label="fit")

                    for j in range(4):
                        test_pars[j] = pars[j]
                        if j == 1:
                            test_pars[j]+=14
                        if j == 2:
                            test_pars[j]+=2.5
                        fit_pars[sec][hist_idx][j] = pars[j]
                    #if hist_idx == paddle_expand:
                    #    test_pars[2]+=3
                    cut_lines[sec][hist_idx][0] = super_gaus_cut_bound(frac_cut,0,*pars)
                    cut_lines[sec][hist_idx][1] = super_gaus_cut_bound(frac_cut,1,*pars)
                    plt.axvline(x=cut_lines[sec][hist_idx][0],color='r',label="Frac:{} low cut ".format(frac_cut))
                    plt.axvline(x=cut_lines[sec][hist_idx][1],color='r',label="Frac:{} top cut ".format(frac_cut))
                except RuntimeError:
                    print("Runtime Error")
                except ValueError:
                    print("Value Error")
            plt.title("Pi+ SC Geometric Distribution Sec:{} Paddle:{}".format(sec+1,hist_idx+1))
            plt.ylabel("Integrated X Yield")
            plt.xlabel("X Sector Centered")
            plt.legend()
            plt.show()



In [ ]:
right_cut_pars = [[cut_lines[i][1][0],3.**(1/2)] for i in range(7)]
left_cut_pars = [[cut_lines[i][1][0],-3.**(1/2)] for i in range(7)]

print(right_cut_pars)
print(right_cut_pars[0])

cont_x_r = np.linspace(-50.0,150.,200)
cont_x_l = np.linspace(-150.0,50.,200)

for sec in range(6):
    plt.hist2d(dumb1_x[sec][cut_hist_len-1], dumb1_y[sec][cut_hist_len-1], bins=(xdim, ydim), weights=dense1[sec][cut_hist_len-1])
    plt.plot(cont_x_r,Poly(cont_x_r,right_cut_pars[sec]),label="right_edge")
    plt.plot(cont_x_l,Poly(cont_x_l,left_cut_pars[sec]),label="left_edge")
    plt.title("Pi+ SC Sector Centered Distribution Sec:{} All Segments and Sides".format(sec+1))
    plt.xlabel("Distance from Center of Detector (cm)")
    plt.ylabel("Distance from the Beamline (cm)")
    plt.show()

In [ ]:
for sec in range(6):
    for hist_idx in range(cut_hist_len):
        plt.hist2d(dumb1_x[sec][hist_idx], dumb1_y[sec][hist_idx], bins=(xdim, ydim), weights=dense1[sec][hist_idx])
        if cut_lines[sec][hist_idx][0] > 0.0:
            plt.axhline(y=cut_lines[sec][hist_idx][0],color='r')
            plt.axhline(y=cut_lines[sec][hist_idx][1],color='r')
        plt.plot(cont_x_r,Poly(cont_x_r,right_cut_pars[sec]),label="right_edge")
        plt.plot(cont_x_l,Poly(cont_x_l,left_cut_pars[sec]),label="left_edge")
        plt.title("Pi+ SC Sector Centered Distribution Sec:{} Paddle:{}".format(sec+1,hist_idx+1))
        plt.xlabel("Distance from Center of Detector (cm)")
        plt.ylabel("Distance from the Beamline (cm)")
        plt.show()